In [44]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
X = np.array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
              [1, 1, 2, 1, 3, 0, 5, 10, 1, 2],
              [500, 700, 750, 600, 1450,
               800, 1500, 2000, 450, 1000],
              [1, 1, 2, 1, 2, 
               1, 3, 3, 1, 2]], dtype = np.float64)
y = np.array([0, 0, 1, 0, 1,
              0, 1, 0, 1, 1], dtype = np.float64)
X

array([[1.00e+00, 1.00e+00, 1.00e+00, 1.00e+00, 1.00e+00, 1.00e+00,
        1.00e+00, 1.00e+00, 1.00e+00, 1.00e+00],
       [1.00e+00, 1.00e+00, 2.00e+00, 1.00e+00, 3.00e+00, 0.00e+00,
        5.00e+00, 1.00e+01, 1.00e+00, 2.00e+00],
       [5.00e+02, 7.00e+02, 7.50e+02, 6.00e+02, 1.45e+03, 8.00e+02,
        1.50e+03, 2.00e+03, 4.50e+02, 1.00e+03],
       [1.00e+00, 1.00e+00, 2.00e+00, 1.00e+00, 2.00e+00, 1.00e+00,
        3.00e+00, 3.00e+00, 1.00e+00, 2.00e+00]])

In [7]:
def calc_std_feat(x):
  res = (x - x.mean()) / x.std()
  return res

X_st = X.copy()
X_st[2, :] = calc_std_feat(X[2, :])
X_st

array([[ 1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
         1.        ,  1.        ,  1.        ,  1.        ,  1.        ],
       [ 1.        ,  1.        ,  2.        ,  1.        ,  3.        ,
         0.        ,  5.        , 10.        ,  1.        ,  2.        ],
       [-0.97958969, -0.56713087, -0.46401617, -0.77336028,  0.97958969,
        -0.36090146,  1.08270439,  2.11385144, -1.08270439,  0.05155735],
       [ 1.        ,  1.        ,  2.        ,  1.        ,  2.        ,
         1.        ,  3.        ,  3.        ,  1.        ,  2.        ]])

In [9]:
def sigmoid(z):
  res = 1 / (1 + np.exp(-z))
  return res

In [3]:
def calc_logloss(y, y_pred):
  err = - np.mean(y * np.log(y_pred) + (1.0 - y) * np.log(1.0 - y_pred))
  err = np.sum(err)
  return err

In [33]:
def eval_model(X, y, iterations, alpha=1e-4):
  np.random.seed(42)
  W = np.random.randn(X.shape[0])
  n = X.shape[1]
  for i in range(1, iterations+1):
    z = np.dot(W, X)
    y_pred = sigmoid(z)
    err = calc_logloss(y, y_pred)
    W -= alpha * (1/n * np.dot((y_pred - y), X.T))
    if i % (iterations / 10) == 0:
      print(i, W, err)
  return W

1*. Измените функцию calc_logloss так, чтобы нули по возможности не попадали в np.log.  

Можно исключить из расчета нулевые значения

In [28]:
def calc_logloss(y, y_pred):
    err = - np.mean(y * np.log(y_pred, where=(y_pred!=0)) + (1.0 - y) * np.log(1.0 - y_pred, where=(1-y_pred!=0)))
    err = np.sum(err)
    return err

2. Подберите аргументы функции eval_model для логистической регрессии таким образом, чтобы log loss был минимальным.

In [34]:
#Методом подбора ошибка меньше 0.00023
W = eval_model(X_st, y, iterations=500000, alpha=50)

50000 [-356.84243808  -24.07878835 -106.64648128  270.98488703] 0.0008434794651377027
100000 [-374.12457107  -25.15845553 -111.88444204  283.94554874] 0.0006441784555743443
150000 [-387.72738028  -26.00834299 -116.00718148  294.14707587] 0.0005209867322289447
200000 [-398.94459207  -26.70922305 -119.40685558  302.55959095] 0.00043731528375534823
250000 [-408.48883729  -27.30559681 -122.29946983  309.71749   ] 0.00037678297766799635
300000 [-416.79466335  -27.82460378 -124.8167365   315.94664391] 0.0003309607124957364
350000 [-424.14666808  -28.28402014 -127.04491621  321.46047856] 0.00029506918408203685
400000 [-430.74141149  -28.69612409 -129.04358482  326.40640025] 0.00026619671396476357
450000 [-436.72034887  -29.06975218 -130.85561587  330.89049162] 0.0002424683101648727
500000 [-442.18867535  -29.41147618 -132.51289256  334.99164309] 0.00022262206468492748


In [35]:
#Изменим функцию, чтобы ее использовать для подбора параметров в цикле

def eval_model_b(X, y, iterations, alpha=1e-4):
    np.random.seed(42)
    W = np.random.randn(X.shape[0])
    n = X.shape[1]
    for i in range(1, iterations+1):
        y_pred = sigmoid(np.dot(W, X))
        err = calc_logloss(y, y_pred)
        W -= alpha * (1/n * np.dot((y_pred - y), X.T))
    return W, err, alpha, iterations

#Подберем параметры, расширив интервалы подбора:

best_params = None
err_min = 10
i = 0
for iterations in [5000, 50000, 100000, 500000]:
    for alpha in [1e-1, 1, 10, 30, 60, 90]:
        params = W, err, alpha, iterations = eval_model_b(X_st, y, iterations=iterations, alpha=alpha)
        if err < err_min:
            err_min = err
            best_params = params

print(f'best params: iterations = {best_params[-1]}, alpha = {best_params[-2]}')

best params: iterations = 500000, alpha = 90


3. Создайте функцию calc_pred_proba, возвращающую предсказанную вероятность класса 1 (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred_proba).

In [36]:
W = eval_model(X_st, y, iterations=500000, alpha=90)

50000 [-561.78641053  -81.69562858 -150.21303124  488.60340285] 8.660673874508308e-05
100000 [-566.60706859  -80.7711192  -155.89668183  486.65504902] 6.548323994207742e-05
150000 [-571.00952572  -77.9695609  -158.22549593  485.85459707] 5.807555745400122e-05
200000 [-574.93972186  -75.46853231 -160.30449186  485.14001326] 5.217335162397066e-05
250000 [-578.48922471  -73.20976211 -162.18210796  484.49464693] 4.736003719176905e-05
300000 [-581.72530896  -71.15043938 -163.89393143  483.90626617] 4.335974704661886e-05
350000 [-584.69885625  -69.25818506 -165.46687778  483.36561968] 3.998254068447416e-05
400000 [-587.44928324  -67.50791595 -166.92179773  482.86554074] 3.709336666698602e-05
450000 [-590.00774663  -65.87980514 -168.27517262  482.40036445] 3.4593571718354854e-05
500000 [-592.39930109  -64.35790883 -169.54025591  481.96553538] 3.2409412842893146e-05


In [38]:
def calc_pred_proba(X, W):
    y_pred_proba = np.floor(sigmoid(np.dot(W, X)))
    return y_pred_proba

In [39]:
calc_pred_proba(X, W)

array([0., 0., 1., 0., 0., 1., 0., 0., 0., 1.])

4. Создайте функцию calc_pred, возвращающую предсказанный класс (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred).

In [40]:
def calc_pred(W, X):
    m = X.shape[1]
    y_pred = np.zeros((1, m), dtype = np.int64)
    y_pred_proba = sigmoid(W.T @ X)
    
    for i in range(len(y_pred_proba)):
        if (y_pred_proba[i] > 0.5): 
            y_pred[:, i] = 1
        elif (y_pred_proba[i] <= 0.5):
            y_pred[:, i] = 0
    return y_pred

In [41]:
y_pred = calc_pred(W, X)
y_pred

array([[0, 0, 1, 0, 0, 1, 0, 0, 0, 1]], dtype=int64)

5. Посчитайте Accuracy, матрицу ошибок, точность и полноту, а также F1 score.

In [42]:
#Accuracy

def accuracy_(y_pred, y):
    return len(y_pred[y == y_pred]) / len(y_pred)

accuracy_(y_pred, y)

6.0

In [45]:
#Correlation matrix

def correlation_matrix(y_pred, y):
    TP = len(y_pred[(y == y_pred) & (y_pred == 1)])
    FN = len(y_pred[(y != y_pred) & (y_pred == 0)])
    FP = len(y_pred[(y != y_pred) & (y_pred == 1)])
    TN = len(y_pred[(y == y_pred) & (y_pred == 0)])
    corr_matrix = pd.DataFrame.from_dict(
        {'y_pred': ['a(x) = +1', 'a(x) = -1'], 
         'y = +1': [TP, FN], 
         'y = -1': [FP, TN]}).set_index('y_pred')
    return corr_matrix


correlation_matrix(y_pred, y)

,y = +1,y = -1
y_pred,,
a(x) = +1,2,1
a(x) = -1,3,4


In [46]:
#Precision:


def precision_(y_pred, y):
    return len(y_pred[(y == y_pred) & (y_pred == 1)]) / (
        len(y_pred[(y == y_pred) & (y_pred == 1)]) + len(y_pred[(y != y_pred) & (y_pred == 1)]))


precision_(y_pred, y)

0.6666666666666666

In [47]:
#Recall:

def recall_(y_pred, y):
    return len(y_pred[(y == y_pred) & (y_pred == 1)]) / (
        len(y_pred[(y == y_pred) & (y_pred == 1)]) + len(y_pred[(y != y_pred) & (y_pred == 0)]))


recall_(y_pred, y)

0.4

In [48]:
#F1-score:

def F1_(y_pred, y):
    return 2 * precision_(y_pred, y) * recall_(y_pred, y) / (precision_(y_pred, y) + recall_(y_pred, y))

F1_(y_pred, y)

0.5

6. Могла ли модель переобучиться? Почему?

Модель переобучилась. Мало данных. Данные не разбиваются на тест и трейн.

7*. Создайте функции eval_model_l1 и eval_model_l2 с применением L1 и L2 регуляризаций соответственно.

In [54]:
#L1
def eval_model_L1(X, y, iterations, alpha=1e-4, lambda_=1e-6):
    np.random.seed(42)
    W = np.random.randn(X.shape[0])
    n = X.shape[1]
    for i in range(1, iterations+1):
        y_pred = sigmoid(np.dot(W, X))
        err = calc_logloss(y, y_pred)
        W -= alpha * (1/n * np.dot((y_pred - y), X.T)  + lambda_ * np.sign(W))
        if i % (iterations / 10) == 0:
            print(i, W, err)
    return W

eval_model_L1(X, y, iterations=1000, alpha=1e-4, lambda_=1e-6)

100 [ 0.49569862 -0.14048597  0.04102277  1.5229336 ] 2.413388514081233
200 [ 0.49526047 -0.14104544  0.03833593  1.52386771] 5.108985887692792
300 [ 0.49482224 -0.14160609  0.03509312  1.52480096] 1.2825931518988793
400 [ 0.4943942  -0.14215142  0.03949183  1.52574929] 1.827573274714005
500 [ 0.49395168 -0.14271136  0.03633122  1.52668127] 1.3874304546812932
600 [ 0.4935234  -0.1432559   0.04103482  1.52762996] 2.4250827017168426
700 [ 0.4930729  -0.14382518  0.03333012  1.52855247] 1.164812383928454
800 [ 0.49264338 -0.14436781  0.03878259  1.52950186] 1.6924975873590777
900 [ 0.49221128 -0.14491816  0.04057558  1.53044482] 3.688873589459651
1000 [ 0.4917702  -0.14548102  0.03606936  1.53137513] 1.3637880016750625


array([ 0.4917702 , -0.14548102,  0.03606936,  1.53137513])

In [60]:
#L2

def eval_model_L2(X, y, iterations, alpha=1e-6, lambda_=1e-7):
    np.random.seed(42)
    W = np.random.randn(X.shape[0])
    n = X.shape[1]
    for i in range(1, iterations+1):
        y_pred = sigmoid(np.dot(W, X))
        err = calc_logloss(y, y_pred)
        W -= alpha * (1/n * np.dot((y_pred - y), X.T) + lambda_ * W)
        if i % (iterations / 10) == 0:
            print(i, W, err)
    return W

eval_model_L2(X, y, iterations=1000, alpha=1e-5, lambda_=1e-6)

100 [ 0.49621415 -0.1395643   0.18768854  1.52232985] 0.0
200 [ 0.49597158 -0.14012408 -0.00414987  1.52207922] 0.8479439031942922
300 [ 0.49591265 -0.14015326 -0.00414983  1.52215009] 0.8479269470644549
400 [ 0.49585371 -0.14018244 -0.00414978  1.52222095] 0.8479099925439716
500 [ 0.49579478 -0.14021162 -0.00414974  1.52229182] 0.8478930396324955
600 [ 0.49573584 -0.14024078 -0.0041497   1.52236268] 0.8478760883296823
700 [ 0.4956769  -0.14026994 -0.00414966  1.52243354] 0.8478591386351848
800 [ 0.49561797 -0.1402991  -0.00414962  1.5225044 ] 0.8478421905486584
900 [ 0.49555903 -0.14032825 -0.00414957  1.52257526] 0.8478252440697567
1000 [ 0.49550009 -0.14035739 -0.00414953  1.52264612] 0.8478082991981344


array([ 0.49550009, -0.14035739, -0.00414953,  1.52264612])